In [1]:
import hls4ml
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-10-03 17:25:01.128630: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-03 17:25:01.129856: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/carolinejohnson/miniconda3/envs/weird/lib/python3.8/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [2]:
from tensorflow.keras.models import Sequential, model_from_json

In [3]:
json_file = open('encoder_8x8_c8_S2_tele.json', 'r')
loaded_model_json = json_file.read()

In [4]:
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('encoder_8x8_c8_S2_tele.h5')
loaded_model.get_weights()
loaded_model.summary()

2023-10-03 17:25:30.358095: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-10-03 17:25:30.389587: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-03 17:25:30.389628: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-03 17:25:30.389680: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist
2023-10-03 17:25:30.392776: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 1)]         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 8)           80        
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
encoded_vector (Dense)       (None, 16)                2064      
Total params: 2,144
Trainable params: 2,144
Non-trainable params: 0
_________________________________________________________________


In [5]:
# #loaded_model.load_weights('encoder_8x8_c8_S2_tele.h5')
# x = np.array([[[[0, 0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[ 0,   0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[ 0,  0,  0,  0,
#                 0, 0,  0,  0 ]]],
 
 
#         [[[ 0,  0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[0,  0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[0,  0,  0,  0,
#                 0, 0,  0,  0 ]]],
 
 
#         [[[0,  0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[0 , 0,  0,  0,
#                 0, 0,  0,  0 ]],
 
#          [[0,  0,  0,  0,
#                 0, 0,  0,  0 ]]]],
#       dtype="float32")
# y = np. array([ 0,  0,  0,  0,  0,
#         0, 0,  0], dtype="float32")
# #loaded_model.summary()
# l = []
# l.append(x)
# l.append(y)
# loaded_model.layers[1].set_weights(l)
# loaded_model.get_weights()

In [16]:
#First, the baseline model
hls_config = hls4ml.utils.config_from_keras_model(loaded_model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['Model']['ReuseFactor'] = 1

# Create an entry for each layer, here you can for instance change the strategy for a layer to 'resource' 
# or increase the reuse factor individually for large layers.
# In this case, we designed the model to be small enough for a fully parallel implementation 
# so we use the latency strategy and reuse factor of 1 for all layers.
for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 3
#If you want best numerical performance for high-accuray models, while the default latency strategy is faster but numerically more unstable
#hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'
#plotting.print_dict(hls_config)

hls_config['LayerName']["encoded_vector"]['ReuseFactor'] = 2

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = loaded_model
cfg['OutputDir']  = 'testingIOType'
cfg['XilinxPart'] = 'xcu250-figd2104-2L-e'
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: conv2d, layer type: Conv2D
  -> Activation (relu), layer name: conv2d
Layer name: encoded_vector, layer type: Dense
  -> Activation (relu), layer name: encoded_vector
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 8, 8, 1]], output shape: [None, 8, 8, 1]
Layer name: conv2d, layer type: Conv2D, input shapes: [[None, 8, 8, 1]], output shape: [None, 4, 4, 8]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 4, 4, 8]], output shape: [None, 128]
Layer name: encoded_vector, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 16]
Creating HLS model
Writing HLS project
Done


In [17]:
loaded_model.summary()
loaded_model.get_weights()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 1)]         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 8)           80        
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
encoded_vector (Dense)       (None, 16)                2064      
Total params: 2,144
Trainable params: 2,144
Non-trainable params: 0
_________________________________________________________________


[array([[[[ 1.43463641e-01, -5.92209935e-01,  1.12317359e+00,
           -2.12841667e-03,  2.08273139e-02,  5.73347986e-01,
            8.00394937e-02,  1.12851836e-01]],
 
         [[ 2.47702882e-01, -3.82787645e-01,  8.97283316e-01,
            4.29426938e-01, -4.47546810e-01, -4.98714261e-02,
            5.73822796e-01, -4.58222851e-02]],
 
         [[-1.36033118e-01,  4.74251397e-02,  4.58531491e-02,
            6.99015036e-02, -1.37546152e-01,  2.65467882e-01,
            2.52627492e-01, -3.66227739e-02]]],
 
 
        [[[-1.77963480e-01, -6.15566432e-01,  1.02063775e+00,
            3.40583533e-01, -2.27287635e-01,  3.44268158e-02,
            4.10290547e-02,  2.89543450e-01]],
 
         [[-2.18355134e-02, -1.94294661e-01,  7.73878455e-01,
            9.81672823e-01, -8.73244047e-01,  4.17940207e-02,
            3.96928005e-02, -1.99224949e-01]],
 
         [[ 4.33081314e-02,  1.86833858e-01, -1.83565430e-02,
            6.04253769e-01, -5.46070933e-01,  5.63281894e-01,
        

In [18]:
cfg['HLSConfig']['LayerName']['conv2d']['IOType'] = 'io_parallel'
cfg['HLSConfig']['Strategy'] = 'Latency'
cfg

{'OutputDir': 'testingIOType',
 'ProjectName': 'myproject',
 'Backend': 'Vivado',
 'XilinxPart': 'xcu250-figd2104-2L-e',
 'Board': None,
 'ClockPeriod': 5,
 'IOType': 'io_stream',
 'HLSConfig': {'Model': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'Strategy': 'Latency'},
  'LayerName': {'input_1': {'Precision': {'result': 'ap_fixed<16,6>'},
    'Strategy': 'Resource',
    'ReuseFactor': 3},
   'conv2d': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 3,
    'Strategy': 'Resource',
    'IOType': 'io_parallel'},
   'conv2d_relu': {'Precision': 'ap_fixed<16,6>',
    'ReuseFactor': 3,
    'table_size': 1024,
    'table_t': 'ap_fixed<18,8>',
    'Strategy': 'Resource'},
   'encoded_vector': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 2,
    'Strategy': 'Resource'},
   'encoded_vector_relu': {'Precision': 'ap_fixed<16,6>

In [10]:
cfg

{'OutputDir': 'testingRegular',
 'ProjectName': 'myproject',
 'Backend': 'Vivado',
 'XilinxPart': 'xcu250-figd2104-2L-e',
 'Board': None,
 'ClockPeriod': 5,
 'IOType': 'io_stream',
 'HLSConfig': {'Model': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'Strategy': 'Latency'},
  'LayerName': {'input_1': {'Precision': {'result': 'ap_fixed<16,6>'},
    'Strategy': 'Resource',
    'ReuseFactor': 3},
   'conv2d': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 3,
    'Strategy': 'Resource'},
   'conv2d_relu': {'Precision': 'ap_fixed<16,6>',
    'ReuseFactor': 3,
    'table_size': 1024,
    'table_t': 'ap_fixed<18,8>',
    'Strategy': 'Resource'},
   'encoded_vector': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 2,
    'Strategy': 'Resource'},
   'encoded_vector_relu': {'Precision': 'ap_fixed<16,6>',
    'ReuseFactor': 3,
   

In [19]:
hls_model.build(csim=False)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2.1 (64-bit)
  **** SW Build 2729669 on Thu Dec  5 04:48:12 MST 2019
  **** IP Build 2729494 on Thu Dec  5 07:38:25 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'carolinejohnson' on host 'ubuntu' (Linux_x86_64 version 5.4.0-150-generic) on Tue Oct 03 17:30:17 PDT 2023
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/carolinejohnson/mkconfig/testingIOType'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/carolinejohnson/mkconfig/testingIOType/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to the project
INFO: [HLS 200-

{'EstimatedClockPeriod': '4.299',
 'BestLatency': '568',
 'WorstLatency': '569',
 'IntervalMin': '163',
 'IntervalMax': '568',
 'BRAM_18K': '467',
 'DSP48E': '1048',
 'FF': '21470',
 'LUT': '26768',
 'URAM': '0',
 'AvailableBRAM_18K': '5376',
 'AvailableDSP48E': '12288',
 'AvailableFF': '3456000',
 'AvailableLUT': '1728000',
 'AvailableURAM': '1280'}

In [9]:
cfg

{'OutputDir': 'testingRegular',
 'ProjectName': 'myproject',
 'Backend': 'Vivado',
 'XilinxPart': 'xcu250-figd2104-2L-e',
 'Board': None,
 'ClockPeriod': 5,
 'IOType': 'io_stream',
 'HLSConfig': {'Model': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'Strategy': 'Latency'},
  'LayerName': {'input_1': {'Precision': {'result': 'ap_fixed<16,6>'},
    'Strategy': 'Resource',
    'ReuseFactor': 3},
   'conv2d': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 3,
    'Strategy': 'Resource'},
   'conv2d_relu': {'Precision': 'ap_fixed<16,6>',
    'ReuseFactor': 3,
    'table_size': 1024,
    'table_t': 'ap_fixed<18,8>',
    'Strategy': 'Resource'},
   'encoded_vector': {'Precision': {'weight': 'ap_fixed<16,6>',
     'bias': 'ap_fixed<16,6>',
     'result': 'ap_fixed<16,6>'},
    'ReuseFactor': 2,
    'Strategy': 'Resource'},
   'encoded_vector_relu': {'Precision': 'ap_fixed<16,6>',
    'ReuseFactor': 3,
   